In [1]:
import os
import numpy as np
import pandas as pd
import random
import time
import scanpy as sc
import torch

import warnings
warnings.filterwarnings('ignore')
import logging
logging.basicConfig(level=logging.INFO)
import scDualGN

def seed_torch(seed=666):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = False
seed_torch()

In [2]:
adata = sc.read_h5ad('/media/bgi/zhouruilong/paper_test/cluster/our_model/COVID-immune/haniffa21.processed.h5ad')
adata.obs['celltype'] = adata.obs['initial_clustering']
adata.obs['celltype_str'] = adata.obs['celltype']
adata.obs['celltype'].cat.categories = list(range(0,18))
adata.obs['celltype'] = adata.obs['celltype'].astype(np.int)

In [3]:
adata

AnnData object with n_obs × n_vars = 647366 × 24929
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id', 'celltype', 'celltype_str'
    var: 'feature_types'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw'

In [4]:
sc.pp.highly_variable_genes(adata, n_top_genes=2500)
adata = adata[:, adata.var.highly_variable]
sc.pp.scale(adata,max_value=10)

In [5]:
adata

AnnData object with n_obs × n_vars = 647366 × 2500
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id', 'celltype', 'celltype_str'
    var: 'feature_types', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw'

In [ ]:
#kmeans
start = time.time()
z_final, y_pred_lab, model = scDualGN.run_scDualGN(adata, n_cluster=18,n_z=32,n_epoch_update_pq=5, n_epochs=30,alpha_dualvae=0.02,beta_daulvae=5,gamma_dualvae=4,GPU_id=2)
end = time.time()

print('running time = {}'.format(end-start))

INFO:scDualGN.run:device: cuda:2
INFO:scDualGN.run:pretrain device: cuda:2
INFO:scDualGN.run:start pretraining...
INFO:scDualGN.run:Epoch 0/50,Overall loss:5789.7701,MSE:954.5647,MSE1:254.1282,KL: 21.6925
INFO:scDualGN.run:Epoch 1/50,Overall loss:3854.5931,MSE:716.1839,MSE1:68.2153,KL: 40.6187
INFO:scDualGN.run:Epoch 2/50,Overall loss:3463.4691,MSE:666.4035,MSE1:32.6053,KL: 51.5005
INFO:scDualGN.run:Epoch 3/50,Overall loss:3337.6647,MSE:651.4998,MSE1:19.7436,KL: 59.5530
INFO:scDualGN.run:Epoch 4/50,Overall loss:3274.9021,MSE:643.9032,MSE1:13.5157,KL: 66.1787
INFO:scDualGN.run:Epoch 5/50,Overall loss:3235.5848,MSE:638.8048,MSE1:10.0309,KL: 71.8777
INFO:scDualGN.run:Epoch 6/50,Overall loss:3204.1925,MSE:634.1273,MSE1:8.0053,KL: 76.7357
INFO:scDualGN.run:Epoch 7/50,Overall loss:3176.3549,MSE:629.5654,MSE1:6.7266,KL: 81.0612
INFO:scDualGN.run:Epoch 8/50,Overall loss:3154.4982,MSE:625.9339,MSE1:5.7807,KL: 85.3138
INFO:scDualGN.run:Epoch 9/50,Overall loss:3132.9700,MSE:622.2236,MSE1:5.0157,K

In [ ]:
#leiden
start = time.time()
z_final, y_pred_lab, model = scDualGN.run_scDualGN(adata,n_z=32,n_epoch_update_pq=5, n_epochs=30,alpha_dualvae=0.02,beta_daulvae=5,gamma_dualvae=4,GPU_id=2,n_neighbors=50)
end = time.time()

print('running time = {}'.format(end-start))